In [180]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
folds = kf.split(range(len(y)))
S_train_i = np.zeros(len(y))
print len(y)
for j, (train_idx, test_idx) in enumerate(folds):
    print (train_idx, test_idx)

30396
(array([    1,     2,     3, ..., 30393, 30394, 30395]), array([    0,     5,    10, ..., 30385, 30388, 30392]))
(array([    0,     1,     2, ..., 30391, 30392, 30395]), array([    9,    16,    18, ..., 30382, 30393, 30394]))
(array([    0,     1,     2, ..., 30393, 30394, 30395]), array([    4,     6,     7, ..., 30377, 30384, 30389]))
(array([    0,     4,     5, ..., 30393, 30394, 30395]), array([    1,     2,     3, ..., 30378, 30390, 30391]))
(array([    0,     1,     2, ..., 30392, 30393, 30394]), array([   15,    20,    26, ..., 30386, 30387, 30395]))


In [155]:
ID = np.array([1,2,3])

In [154]:
df = pd.DataFrame({'id':[1,2], 'y':[4,5]})

In [157]:
df2 = pd.DataFrame(ID, columns=['id'])

In [158]:
df2.merge(df, on='id', how='left')

,id,y
0,1,4.0
1,2,5.0
2,3,NaN


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
from sklearn import ensemble   
from sklearn import datasets   
from sklearn.utils import shuffle   
import xgboost as xgb
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)

In [1]:
class XgbModel(object):
    def __init__(self, params, num_boost_rounds):
        self.params = params
        self.num_boost_rounds = num_boost_rounds
        
    def copy(self):
        return XgbModel(self.params, self.num_boost_rounds)
    
    def fit(self, X, y):
        xgtrain = xgb.DMatrix(X, y)
        self.model = xgb.train(self.params, xgtrain, self.num_boost_rounds)
        
    def predict(self, X):
        xgtest = xgb.DMatrix(X)
        return self.model.predict(xgtest)

In [267]:
from stacking import Stacking

In [275]:
import stacking

In [276]:
reload(stacking)

<module 'stacking' from 'stacking.py'>

In [3]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [56]:
def logrmse(predictions, targets):
    return rmse(np.log(predictions), np.log(targets))

# bruno

In [238]:
import bruno

In [239]:
reload(bruno)

<module 'bruno' from 'bruno.pyc'>

In [279]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 420  # From Bruno's original CV, I think
bruno_xgb = XgbModel(xgb_params, num_boost_rounds)

# gunja

In [241]:
import gunja

In [242]:
reload(gunja)

<module 'gunja' from 'gunja.pyc'>

In [280]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 422
gunja_xgb = XgbModel(xgb_params, num_boost_rounds)

# louis

In [244]:
import louis

In [245]:
reload(louis)

<module 'louis' from 'louis.pyc'>

In [281]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 385  # This was the CV output, as earlier version shows
louis_xgb = XgbModel(xgb_params, num_boost_rounds)

# stacking

In [250]:
class DataResolver(object):
    def __init__(self):
        self.__time = 0
    
    def next(self):
        if (self.__time >= 2):
            return louis.resolver()
        self.__time = self.__time + 1
        if (self.__time >= 1):
            return gunja.resolver()
        return bruno.resolver()

In [283]:
base_models = []
base_models.append(bruno_xgb)
base_models.append(gunja_xgb)
base_models.append(louis_xgb)
stacking = Stacking(3, base_models, DataResolver(), logrmse)

In [ ]:
s_train, s_test = stacking.fit(id, 7662)#38132 30471

model 0
(array([    1,     2,     3, ..., 30393, 30394, 30395]), array([    0,     5,     9, ..., 30385, 30388, 30392]))


In [229]:
id, X, y, T = gunja.resolver()

In [187]:
id, X, y, T = louis.resolver()

In [134]:
X, y, T = bruno.Resolver().get()

(38132, 391)
(38132L, 395L)
(38132L, 395L)


In [101]:
louis_xgb.fit(X, y)

In [102]:
y_pred = louis_xgb.predict(X)

In [103]:
logrmse(y, y_pred)

2.9603244826493116